In [ ]:
%matplotlib inline

import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import tree
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

from sklearn.naive_bayes import GaussianNB

from sklearn.svm import SVC

from sklearn.neural_network import MLPRegressor
from sklearn.multioutput import MultiOutputRegressor


import sqlite3
connection = sqlite3.connect("./database/test.db")

In [ ]:
oneHotEncoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
labelEncoder = LabelEncoder()

## 1º Try - The most basic model possible

This was the first model we created, and it served as a foundation for testing various machine learning algorithms. We only chose the **team ID**, **player ID**, **year**, and **playoff** (whether the team made the playoffs or not) for this model.

The data was divided into training (the first nine years of data) and test sets (the tenth year of data). Also, due to the inclusion of the player IDs, we aren’t determining which teams go to the playoffs, but which players go instead. This type of division into training and test sets was also used in the next models.


In [ ]:
query = "select teams.tmID as tmID, playerID, teams.year as year, playoff from players_teams join teams \
on teams.tmID = players_teams.tmID and teams.year = players_teams.year order by teams.year, teams.name ;"

df = pd.read_sql(query, connection)
categorical_columns = ["tmID", "playerID"]

for col in categorical_columns:
    df[col] = labelEncoder.fit_transform(df[col].astype('str'))
    
# replace 'Y' and 'N' with 1 and 0 in the playoff column
df["playoff"] = df["playoff"].replace("Y", 1)
df["playoff"] = df["playoff"].replace("N", 0)
    
print(df.head())
    
# split the data into train and test

train_df = df[df["year"] < 10]

train_inputs = train_df.iloc[:, :-1].values
train_labels = train_df.iloc[:, -1].values
    
test_df = df[df["year"] == 10]

print(test_df)

test_inputs = test_df.iloc[:, :-1].values
test_labels = test_df.iloc[:, -1].values


For this model, we decided to test it with the four different ML algorithms:



### Decision Tree Algorithm

In [ ]:
clf = DecisionTreeClassifier(random_state=1)
clf.fit(train_inputs,train_labels)


tree.plot_tree(clf)

# metrics for the decision tree
y_pred = clf.predict(test_inputs)
print("Accuracy:",metrics.accuracy_score(test_labels, y_pred))
print("Precision:",metrics.precision_score(test_labels, y_pred))
print("Recall:",metrics.recall_score(test_labels, y_pred))
print("F1:",metrics.f1_score(test_labels, y_pred))

# confusion matrix
confusion = confusion_matrix(test_labels, y_pred)
print(f"Confusion matrix:\n{confusion}")

#plot confusion matrix
plt.figure(figsize=(5,5))
sb.heatmap(confusion, annot=True, fmt="g", linewidths=.5, square = True, xticklabels=['No', 'Yes'], yticklabels=['No', 'Yes'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
all_sample_title = 'Passed to the playoffs?'
plt.title(all_sample_title, size = 10)
plt.show()

# roc curve
y_pred_proba = clf.predict_proba(test_inputs)[::,1]
fpr, tpr, _ = metrics.roc_curve(test_labels,  y_pred_proba)
auc = metrics.roc_auc_score(test_labels, y_pred_proba)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.legend(loc=4)
plt.show()

### Logistic Regression 

In [ ]:
logreg = LogisticRegression(max_iter=1000)
logreg.fit(train_inputs, train_labels)

y_pred = logreg.predict(test_inputs)

# metrics for the logistic regression
accuracy = accuracy_score(test_labels, y_pred)
print(f"Accuracy: {accuracy:.2f}")
precision = metrics.precision_score(test_labels, y_pred)
print(f"Precision: {precision:.2f}")
recall = metrics.recall_score(test_labels, y_pred)
print(f"Recall: {recall:.2f}")
f1 = metrics.f1_score(test_labels, y_pred)
print(f"F1: {f1:.2f}")

# plot the confusion matrix
confusion = confusion_matrix(test_labels, y_pred)

plt.figure(figsize=(5,5))
sb.heatmap(confusion, annot=True, fmt="g", linewidths=.5, square = True, xticklabels=['No', 'Yes'], yticklabels=['No', 'Yes'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
all_sample_title = 'Passed to the playoffs?'

plt.title(all_sample_title, size = 10)
plt.show()

# roc curve
y_pred_proba = logreg.predict_proba(test_inputs)[::,1]
fpr, tpr, _ = metrics.roc_curve(test_labels,  y_pred_proba)
auc = metrics.roc_auc_score(test_labels, y_pred_proba)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.legend(loc=4)
plt.show()

### Naive Bayes

In [ ]:
# Naive Bayes
gnb = GaussianNB()
gnb.fit(train_inputs, train_labels)

y_pred = gnb.predict(test_inputs)

# metrics for the naive bayes
accuracy = accuracy_score(test_labels, y_pred)
print(f"Accuracy: {accuracy:.2f}")
precision = metrics.precision_score(test_labels, y_pred)
print(f"Precision: {precision:.2f}")
recall = metrics.recall_score(test_labels, y_pred)
print(f"Recall: {recall:.2f}")
f1 = metrics.f1_score(test_labels, y_pred)
print(f"F1: {f1:.2f}")

# plot the confusion matrix
confusion = confusion_matrix(test_labels, y_pred)

plt.figure(figsize=(5,5))
sb.heatmap(confusion, annot=True, fmt="g", linewidths=.5, square = True, xticklabels=['No', 'Yes'], yticklabels=['No', 'Yes'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
all_sample_title = 'Passed to the playoffs?'

plt.title(all_sample_title, size = 10)
plt.show()

# roc curve
y_pred_proba = gnb.predict_proba(test_inputs)[::,1]
fpr, tpr, _ = metrics.roc_curve(test_labels,  y_pred_proba)
auc = metrics.roc_auc_score(test_labels, y_pred_proba)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.legend(loc=4)
plt.show()

### Support Vector Machine (SVM)

In [ ]:
# SVM
svm = SVC(kernel='linear')
svm.fit(train_inputs, train_labels)

y_pred = svm.predict(test_inputs)

# metrics for the SVM
accuracy = accuracy_score(test_labels, y_pred)
print(f"Accuracy: {accuracy:.2f}")
precision = metrics.precision_score(test_labels, y_pred)
print(f"Precision: {precision:.2f}")
recall = metrics.recall_score(test_labels, y_pred)
print(f"Recall: {recall:.2f}")
f1 = metrics.f1_score(test_labels, y_pred)
print(f"F1: {f1:.2f}")

# plot the confusion matrix
confusion = confusion_matrix(test_labels, y_pred)

plt.figure(figsize=(5,5))
sb.heatmap(confusion, annot=True, fmt="g", linewidths=.5, square = True, xticklabels=['No', 'Yes'], yticklabels=['No', 'Yes'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
all_sample_title = 'Passed to the playoffs?'

plt.title(all_sample_title, size = 10)
plt.show()

In this approach all 4 algorithms performed poorly. This was expected considering the data they are trained on is very limited since it does not include important statistics and only if the set player + team went to the playoffs in a given year.

## 2º Try - Team Performance in Consideration

To improve the quality of our predictions in comparison with the previous model we decided to train our model based on the teams average statistics per player from the previous year. 

With this approach we solve the problem of having predictions on the players instead of on the teams.

In [ ]:
## create an empty dataframe without any column names, indices or data
df = pd.DataFrame()

## obtain the team IDS and year
team_ids = pd.read_sql("select tmID, year from teams", connection)

## iterate through each team and year
for index, row in team_ids.iterrows():
    ## obtain the team ID and year for each row
    team_id = row['tmID']
    year = row['year']
    
    if(year == 1):
        continue
    
    query = "select tmID, year + 1 as year, avg(oRebounds), avg(dRebounds), avg(rebounds), avg(assists), avg(steals), avg(blocks), \
        avg(turnovers), avg(PF), avg(fgAttempted), avg(fgMade), avg(ftAttempted), avg(ftMade), avg(threeAttempted), avg(threeMade) \
        from players_teams where tmID = '" + team_id + "' and year = " + str(year - 1) + ";"
        
    ## obtain the average stats for each team
    team_stats = pd.read_sql(query, connection)
    
    query = "select avg(weight), avg(height) from players_teams join players on players.bioID = players_teams.playerID \
        where tmID = '" + team_id + "' and year = " + str(year) + ";"
        
    ## obtain the average weight and height for each team
    team_weight_height = pd.read_sql(query, connection)
    
    ## add the average weight and height to the team stats
    team_stats['weight'] = team_weight_height['avg(weight)']
    team_stats['height'] = team_weight_height['avg(height)']
    
    # transform the NaN values to 0
    team_stats = team_stats.fillna(0)
    
    # include in the query the win and loss ratio
    query = "select tmID, year, won, lost, GP from teams where tmID = '" + team_id + "' and year = " + str(year - 1) + ";"
    
    team_win_loss = pd.read_sql(query, connection)
    
    # obtain the win and loss ratio
    win_ratio = team_win_loss['won'] / team_win_loss['GP']
    loss_ratio = team_win_loss['lost'] / team_win_loss['GP']
    
    # add the win and loss ratio to the team stats
    team_stats['win_ratio'] = win_ratio
    team_stats['loss_ratio'] = loss_ratio
    
    query = "select count(pt.playerID) as num_player_awards from players_teams pt join awards_players ap on pt.playerID = ap.playerID \
        where pt.year = ap.year and pt.tmID = '" + team_id + "' and pt.year < " + str(year) + " group by pt.tmID, ap.year;"
        
    team_awards = pd.read_sql(query, connection)
    
    # add the number of player awards to the team stats
    team_stats['num_player_awards'] = team_awards['num_player_awards']
    
    # transform the NaN values to 0
    team_stats = team_stats.fillna(0)
    
    # obtain the number of coach awards
    query = "select count(coachID) as num_coach_awards from coaches join awards_players ap on coaches.coachID = ap.playerID \
        where coaches.year = ap.year and coaches.tmID = '" + team_id + "' and coaches.year < " + str(year) + " group by coaches.tmID, ap.year;"
        
    team_coach_awards = pd.read_sql(query, connection)
    
    # add the number of coach awards to the team stats
    team_stats['num_coach_awards'] = team_coach_awards['num_coach_awards']
    
    # transform the NaN values to 0
    team_stats = team_stats.fillna(0)
        
    query = "select tmID, year from teams where semis <> '' and tmID = '" + team_id + "' and year = " + str(year - 1) + ";"
    semis = pd.read_sql(query, connection)
    
    # team reached semis
    if semis.empty:
        team_stats['semis'] = 0
    else:
        team_stats['semis'] = 1
        
    query = "select tmID, year from teams where finals <> '' and tmID = '" + team_id + "' and year = " + str(year - 1) + ";"
    finals = pd.read_sql(query, connection)
    
    # team reached finals
    if finals.empty:
        team_stats['finals'] = 0
    else:
        team_stats['finals'] = 1
        
    query = "select playoff from teams where tmID = '" + team_id + "' and year = " + str(year) + ";"
    playoff = pd.read_sql(query, connection)
    
    # check if it's Y or N
    if playoff['playoff'][0] == 'Y':
        team_stats['playoff'] = 1
    else:
        team_stats['playoff'] = 0
        
    
    # append the stats to the dataframe
    df = df._append(team_stats, ignore_index=True)
    
    if(year == 10):
        print(team_stats)

In [ ]:
# encode the categorical columns

team_pairs = []
team_names = df["tmID"].values
df["tmID"] = labelEncoder.fit_transform(df["tmID"].astype('str'))   

team_pairs = set(zip(team_names, df["tmID"].values))

print(team_pairs)   
        
# remove the rows that have year = 10
train_df = df[df.year < 10]

# remove the rows that have year <> 10
test_df = df[df.year == 10]

# create the training labels and inputs
train_inputs = train_df.iloc[:, :-1].values
train_labels = train_df.iloc[:, -1].values

# create the testing labels and inputs
test_inputs = test_df.iloc[:, :-1].values
test_labels = test_df.iloc[:, -1].values

### Decision Tree Classifier

In [ ]:
clf = DecisionTreeClassifier(random_state=1)
clf.fit(train_inputs,train_labels)


tree.plot_tree(clf)

# metrics for the decision tree
y_pred = clf.predict(test_inputs)
print("Accuracy:",metrics.accuracy_score(test_labels, y_pred))
print("Precision:",metrics.precision_score(test_labels, y_pred))
print("Recall:",metrics.recall_score(test_labels, y_pred))
print("F1:",metrics.f1_score(test_labels, y_pred))

# confusion matrix
confusion = confusion_matrix(test_labels, y_pred)
print(f"Confusion matrix:\n{confusion}")

#plot confusion matrix
plt.figure(figsize=(5,5))
sb.heatmap(confusion, annot=True, fmt="g", linewidths=.5, square = True, xticklabels=['No', 'Yes'], yticklabels=['No', 'Yes'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
all_sample_title = 'Passed to the playoffs?'
plt.title(all_sample_title, size = 10)
plt.show()

# roc curve
y_pred_proba = clf.predict_proba(test_inputs)[::,1]
fpr, tpr, _ = metrics.roc_curve(test_labels,  y_pred_proba)
auc = metrics.roc_auc_score(test_labels, y_pred_proba)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.legend(loc=4)
plt.show()


In [ ]:
query = "select tmId, confID from teams where year = 10;"
teams_conf_ids = pd.read_sql(query, connection)

west_teams = teams_conf_ids[teams_conf_ids["confID"] == "WE"]
east_teams = teams_conf_ids[teams_conf_ids["confID"] == "EA"]

# print the probabilities for each class
probs = clf.predict_proba(test_inputs)
probs_west = []
probs_east = []

for i in range(len(probs)):
    team_id = test_df["tmID"].values[i]
    
    # obtain the team name from the team_pairs
    team_name = [x[0] for x in team_pairs if x[1] == team_id]
    
    if team_name in west_teams["tmID"].values:
        probs_west.append((team_name[0], probs[i][1]))
        
    if team_name in east_teams["tmID"].values:
        probs_east.append((team_name[0], probs[i][1]))
    
    # print(f"{team_name[0]}: {probs[i][1]}")

probs_west = sorted(probs_west, key=lambda x: x[1], reverse=True)
probs_east = sorted(probs_east, key=lambda x: x[1], reverse=True)

print(probs_west)
print(probs_east)

### Logistic Regression

In [ ]:
logreg = LogisticRegression(max_iter=2000)
logreg.fit(train_inputs, train_labels)

y_pred = logreg.predict(test_inputs)

# metrics for the logistic regression
accuracy = accuracy_score(test_labels, y_pred)
print(f"Accuracy: {accuracy:.2f}")
precision = metrics.precision_score(test_labels, y_pred)
print(f"Precision: {precision:.2f}")
recall = metrics.recall_score(test_labels, y_pred)
print(f"Recall: {recall:.2f}")
f1 = metrics.f1_score(test_labels, y_pred)
print(f"F1: {f1:.2f}")

# plot the confusion matrix
confusion = confusion_matrix(test_labels, y_pred)

plt.figure(figsize=(5,5))
sb.heatmap(confusion, annot=True, fmt="g", linewidths=.5, square = True, xticklabels=['No', 'Yes'], yticklabels=['No', 'Yes'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
all_sample_title = 'Passed to the playoffs?'

plt.title(all_sample_title, size = 10)
plt.show()

# roc curve
y_pred_proba = logreg.predict_proba(test_inputs)[::,1]
fpr, tpr, _ = metrics.roc_curve(test_labels,  y_pred_proba)
auc = metrics.roc_auc_score(test_labels, y_pred_proba)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.legend(loc=4)
plt.show()

In [ ]:
query = "select tmId, confID from teams where year = 10;"
teams_conf_ids = pd.read_sql(query, connection)

# print(teams_conf_ids)

west_teams = teams_conf_ids[teams_conf_ids["confID"] == "WE"]
east_teams = teams_conf_ids[teams_conf_ids["confID"] == "EA"]

# print(west_teams)
# print(east_teams)

# print the probabilities for each class
probs = logreg.predict_proba(test_inputs)
probs_west = []
probs_east = []

for i in range(len(probs)):
    team_id = test_df["tmID"].values[i]
    
    # obtain the team name from the team_pairs
    team_name = [x[0] for x in team_pairs if x[1] == team_id]
    
    if team_name in west_teams["tmID"].values:
        probs_west.append((team_name[0], probs[i][1]))
        
    if team_name in east_teams["tmID"].values:
        probs_east.append((team_name[0], probs[i][1]))
    
    # print(f"{team_name[0]}: {probs[i][1]}")

probs_west = sorted(probs_west, key=lambda x: x[1], reverse=True)
probs_east = sorted(probs_east, key=lambda x: x[1], reverse=True)

print(probs_west)
print(probs_east)

### Naive Bayes

In [ ]:
# Naive Bayes
gnb = GaussianNB()
gnb.fit(train_inputs, train_labels)

y_pred = gnb.predict(test_inputs)

# metrics for the naive bayes
accuracy = accuracy_score(test_labels, y_pred)
print(f"Accuracy: {accuracy:.2f}")
precision = metrics.precision_score(test_labels, y_pred)
print(f"Precision: {precision:.2f}")
recall = metrics.recall_score(test_labels, y_pred)
print(f"Recall: {recall:.2f}")
f1 = metrics.f1_score(test_labels, y_pred)
print(f"F1: {f1:.2f}")

# plot the confusion matrix
confusion = confusion_matrix(test_labels, y_pred)

plt.figure(figsize=(5,5))
sb.heatmap(confusion, annot=True, fmt="g", linewidths=.5, square = True, xticklabels=['No', 'Yes'], yticklabels=['No', 'Yes'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
all_sample_title = 'Passed to the playoffs?'

plt.title(all_sample_title, size = 10)
plt.show()

# roc curve
y_pred_proba = gnb.predict_proba(test_inputs)[::,1]
fpr, tpr, _ = metrics.roc_curve(test_labels,  y_pred_proba)
auc = metrics.roc_auc_score(test_labels, y_pred_proba)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.legend(loc=4)
plt.show()

In [ ]:
query = "select tmId, confID from teams where year = 10;"
teams_conf_ids = pd.read_sql(query, connection)

# print(teams_conf_ids)

west_teams = teams_conf_ids[teams_conf_ids["confID"] == "WE"]
east_teams = teams_conf_ids[teams_conf_ids["confID"] == "EA"]

# print(west_teams)
# print(east_teams)

# print the probabilities for each class
probs = gnb.predict_proba(test_inputs)
probs_west = []
probs_east = []

for i in range(len(probs)):
    team_id = test_df["tmID"].values[i]
    
    # obtain the team name from the team_pairs
    team_name = [x[0] for x in team_pairs if x[1] == team_id]
    
    if team_name in west_teams["tmID"].values:
        probs_west.append((team_name[0], probs[i][1]))
        
    if team_name in east_teams["tmID"].values:
        probs_east.append((team_name[0], probs[i][1]))
    
    # print(f"{team_name[0]}: {probs[i][1]}")

probs_west = sorted(probs_west, key=lambda x: x[1], reverse=True)
probs_east = sorted(probs_east, key=lambda x: x[1], reverse=True)

print(probs_west)
print(probs_east)

### Support Vector Machines (SVM)

In [ ]:
# SVM
svm = SVC(kernel='linear', probability=True)
svm.fit(train_inputs, train_labels)

y_pred = svm.predict(test_inputs)

# metrics for the SVM
accuracy = accuracy_score(test_labels, y_pred)
print(f"Accuracy: {accuracy:.2f}")
precision = metrics.precision_score(test_labels, y_pred)
print(f"Precision: {precision:.2f}")
recall = metrics.recall_score(test_labels, y_pred)
print(f"Recall: {recall:.2f}")
f1 = metrics.f1_score(test_labels, y_pred)
print(f"F1: {f1:.2f}")

# plot the confusion matrix
confusion = confusion_matrix(test_labels, y_pred)

plt.figure(figsize=(5,5))
sb.heatmap(confusion, annot=True, fmt="g", linewidths=.5, square = True, xticklabels=['No', 'Yes'], yticklabels=['No', 'Yes'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
all_sample_title = 'Passed to the playoffs?'

plt.title(all_sample_title, size = 10)
plt.show()


In [ ]:
query = "select tmId, confID from teams where year = 10;"
teams_conf_ids = pd.read_sql(query, connection)

# print(teams_conf_ids)

west_teams = teams_conf_ids[teams_conf_ids["confID"] == "WE"]
east_teams = teams_conf_ids[teams_conf_ids["confID"] == "EA"]

# print(west_teams)
# print(east_teams)

# print the probabilities for each class
probs = svm.predict_proba(test_inputs)
probs_west = []
probs_east = []

for i in range(len(probs)):
    team_id = test_df["tmID"].values[i]
    
    # obtain the team name from the team_pairs
    team_name = [x[0] for x in team_pairs if x[1] == team_id]
    
    if team_name in west_teams["tmID"].values:
        probs_west.append((team_name[0], probs[i][1]))
        
    if team_name in east_teams["tmID"].values:
        probs_east.append((team_name[0], probs[i][1]))
    
    # print(f"{team_name[0]}: {probs[i][1]}")

probs_west = sorted(probs_west, key=lambda x: x[1], reverse=True)
probs_east = sorted(probs_east, key=lambda x: x[1], reverse=True)

print(probs_west)
print(probs_east)

In this approach we got better results than in the previous one but they are still far from good.

SVM, Logistic Regression and Naive Bayes were able to produce 6 True Positive classifications.

However it still has some limitations, namely the fact that it can't deal with a change in the teams composition and it also does not use information regarding coaches and awards.

# 3rd try - Based on the players perfomance

In a third approach we decided to try to predict the player performance considering their historical data and then consider the teams as a sum of all players.

In [ ]:
df = pd.read_sql("select * from players join players_teams on players.bioID = players_teams.playerID;", connection)

columns = ['bioID', 'pos', 'height', 'weight', 'college', 'collegeOther',
       'birthDate', 'year', 'stint', 'tmID', 'points', 'oRebounds', 'dRebounds', 'rebounds',
       'assists', 'steals', 'blocks', 'turnovers', 'PF', 'fgAttempted',
       'fgMade', 'ftAttempted', 'ftMade', 'threeAttempted', 'threeMade']

df = df[columns]

# get bioID and year from the dataframe
bioID = df["bioID"].values
year = df["year"].values

iterable = zip(bioID, year)

# iterate through the (bioID, year) pairs

for bioID, year in iterable:
        # get number of awards for the player in the team in the year
        query = "select count(award) as num_awards_player from awards_players ap join players_teams pt on ap.year = pt.year \
                and ap.playerID = pt.playerID where ap.playerID = '" + bioID + "' and ap.year <= " + str(year) + ";"
                
        player_awards = pd.read_sql(query, connection)
        
        # if(player_awards["num_awards_player"].values[0] > 0):
        #         print(bioID, year, player_awards)
                
        # add number of awards to the dataframe
        df.loc[(df["bioID"] == bioID) & (df["year"] == year), "num_awards_player"] = player_awards["num_awards_player"].values[0]
     
# extract year from birthDate
df["birthDate"] = pd.to_datetime(df["birthDate"])
df["birthDate"] = df["birthDate"].dt.year

player_ids_10 = df[df["year"] == 10]["bioID"].values

In [ ]:
# transform categorical data
categorical_columns = ['bioID', 'pos', 'college', 'collegeOther', 'tmID']
for col in categorical_columns:
    df[col] = df[col].astype('category')
    
df = pd.get_dummies(df, columns=categorical_columns)

print(df['birthDate'])

# # get all rows from df where year = 10
test_data = df.loc[df["year"] == 10]

# # get all rows from df where year <> 10
train_data = df.loc[df["year"] < 10]


labels = ['points', 'oRebounds', 'dRebounds', 'rebounds', 'assists', 'steals',
       'blocks', 'turnovers', 'PF', 'fgAttempted', 'fgMade', 'ftAttempted',
       'ftMade', 'threeAttempted', 'threeMade']

inputs = []

for col in train_data.columns:
    if col not in labels:
        inputs.append(col)

print(inputs)

train_inputs = train_data[inputs].values
train_labels = train_data[labels].values

test_inputs = test_data[inputs].values
test_labels = test_data[labels].values

# scale data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_inputs = scaler.fit_transform(train_inputs)
test_inputs = scaler.transform(test_inputs)

We predicted each player statistic using a Neural Network and, considering there are multiple target variables, we extended the MLPRegressor with a MultiOutputRegressor to predict all the variables needed.

In [ ]:
# train model
model = MultiOutputRegressor(MLPRegressor(hidden_layer_sizes=(100, 100, 100), 
    max_iter=2000,batch_size=32, alpha=0.0001, solver='adam', verbose=10, random_state=21, tol=0.000000001))
model.fit(train_inputs, train_labels)

# test model
predictions = model.predict(test_inputs)
print(predictions)

In [ ]:
# convert all predictions to integers
for i in range(len(predictions)):
    for j in range(len(predictions[i])):
        predictions[i][j] = int(round(predictions[i][j]))

for i in range(len(predictions)):
    # print("player: ", i)
    print("player: ", player_ids_10[i])
    for j in range(len(predictions[i])):
        print(labels[j], ":  predicted: ", predictions[i][j], " actual: ", test_labels[i][j])
    print("\n")

Now we need to group all players by teams and sum their statistics.

In [ ]:
# get team ids from the dataframe from all years 
team_ids = set(df["tmID"].values)

# create a dictionary where the keys are the team ids and the values are the avg stats for the team
avg_team_stats = {}

for team_id in team_ids:
    avg_team_stats[team_id] = [0] * len(labels)
    
print(test_data.columns)

# iterate through the team ids
for team_id in team_ids:
    
    for i in range(len(predictions)):
 
        if(test_data["tmID_" + team_id].values[i] == 1):
            avg_team_stats[team_id] = [x + y for x, y in zip(avg_team_stats[team_id], predictions[i])]
            
        
# remove tmID from the dictionary that contain all the avg stats for the teams at 0
for team_id in team_ids:
    if(avg_team_stats[team_id] == [0] * len(labels)):
        del avg_team_stats[team_id]
        
        
print(avg_team_stats)

In [ ]:
query = "select * from teams where year < 10;"

team_stats = pd.read_sql(query, connection)

columns = ['tmID', 'year','o_pts', 'o_oreb', 'o_dreb', 'o_reb', 'o_asts', 'o_stl',
       'o_blk', 'o_to', 'o_pf', 'o_fga', 'o_fgm', 'o_fta',
       'o_ftm', 'o_3pa', 'o_3pm', 'confID', 'playoff']

team_stats = team_stats[columns]

query = "select tmID from teams where year = 10;"
team_ids = pd.read_sql(query, connection)
team_ids = team_ids["tmID"].values

team_stats = team_stats.drop(columns=["tmID"])

categorical_columns = ['confID']

for col in categorical_columns:
       team_stats[col] = team_stats[col].astype('category')
       
team_stats = pd.get_dummies(team_stats, columns=categorical_columns)

### Logistic Regression

In [ ]:
# logistic regression
from sklearn.metrics import confusion_matrix

logreg = LogisticRegression(max_iter=1000)
logreg.fit(train_inputs,train_labels)

y_pred = logreg.predict(test_inputs)

print(train_inputs)
print(test_inputs)


#print metrics
print("Accuracy:",metrics.accuracy_score(test_labels,y_pred))
print("Precision:",metrics.precision_score(test_labels, y_pred, pos_label="Y"))
print("Recall:",metrics.recall_score(test_labels, y_pred, pos_label="Y"))
print("F1:",metrics.f1_score(test_labels, y_pred, pos_label="Y"))

# confusion matrix
confusion = confusion_matrix(test_labels, y_pred)
print(f"Confusion matrix:\n{confusion}")

#plot confusion matrix
plt.figure(figsize=(5,5))
sb.heatmap(confusion, annot=True, fmt="g", linewidths=.5, square = True, xticklabels=['No', 'Yes'], yticklabels=['No', 'Yes'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
all_sample_title = 'Passed to the playoffs?'
plt.title(all_sample_title, size = 10)
plt.show()

query = "select tmId, confID from teams where year = 10;"
teams_conf_ids = pd.read_sql(query, connection)

query = "select tmID from teams where year = 10;"

# print(teams_conf_ids)

west_teams = teams_conf_ids[teams_conf_ids["confID"] == "WE"]
east_teams = teams_conf_ids[teams_conf_ids["confID"] == "EA"]

# print(west_teams)
# print(east_teams)

# print the probabilities for each class
probs = logreg.predict_proba(test_inputs)
probs_west = []
probs_east = []

for i in range(len(probs)):
    team_id = team_ids[i]
    
    if team_id in west_teams["tmID"].values:
        probs_west.append((team_id, probs[i][1]))
        
    if team_id in east_teams["tmID"].values:
        probs_east.append((team_id, probs[i][1]))
    
    # print(f"{team_name[0]}: {probs[i][1]}")

probs_west = sorted(probs_west, key=lambda x: x[1], reverse=True)
probs_east = sorted(probs_east, key=lambda x: x[1], reverse=True)

print(probs_west)
print(probs_east)

### Neural Network

In [ ]:
# neural network

from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(100, 100, 100), max_iter=1000, alpha=0.0001, solver='adam', verbose=0, random_state=21, tol=0.000000001)
mlp.fit(train_inputs,train_labels)

y_pred = mlp.predict(test_inputs)
#print metrics
print("Accuracy:",metrics.accuracy_score(test_labels,y_pred))
print("Precision:",metrics.precision_score(test_labels, y_pred, pos_label="Y"))

print("Recall:",metrics.recall_score(test_labels, y_pred, pos_label="Y"))
print("F1:",metrics.f1_score(test_labels, y_pred, pos_label="Y"))

# confusion matrix
confusion = confusion_matrix(test_labels, y_pred)
print(f"Confusion matrix:\n{confusion}")

#plot confusion matrix
plt.figure(figsize=(5,5))
sb.heatmap(confusion, annot=True, fmt="g", linewidths=.5, square = True, xticklabels=['No', 'Yes'], yticklabels=['No', 'Yes'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
all_sample_title = 'Passed to the playoffs?'
plt.title(all_sample_title, size = 10)
plt.show()

# print the probabilities for each class
probs = mlp.predict_proba(test_inputs)
probs_west = []
probs_east = []

for i in range(len(probs)):
    team_id = team_ids[i]
    
    if team_id in west_teams["tmID"].values:
        probs_west.append((team_id, probs[i][1]))
        
    if team_id in east_teams["tmID"].values:
        probs_east.append((team_id, probs[i][1]))
    
    # print(f"{team_name[0]}: {probs[i][1]}")
    
probs_west = sorted(probs_west, key=lambda x: x[1], reverse=True)
probs_east = sorted(probs_east, key=lambda x: x[1], reverse=True)

print(probs_west)
print(probs_east)

### Support Vector Machine (SVM)

In [ ]:
# SVM

from sklearn import svm

clf = svm.SVC(kernel='linear', C=1, probability=True)
clf.fit(train_inputs,train_labels)

y_pred = clf.predict(test_inputs)

#print metrics
print("Accuracy:",metrics.accuracy_score(test_labels,y_pred))
print("Precision:",metrics.precision_score(test_labels, y_pred, pos_label="Y"))
print("Recall:",metrics.recall_score(test_labels, y_pred, pos_label="Y"))
print("F1:",metrics.f1_score(test_labels, y_pred, pos_label="Y"))

# confusion matrix
confusion = confusion_matrix(test_labels, y_pred)
print(f"Confusion matrix:\n{confusion}")

#plot confusion matrix
plt.figure(figsize=(5,5))
sb.heatmap(confusion, annot=True, fmt="g", linewidths=.5, square = True, xticklabels=['No', 'Yes'], yticklabels=['No', 'Yes'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
all_sample_title = 'Passed to the playoffs?'

plt.title(all_sample_title, size = 10)
plt.show()

# print the probabilities for each class
probs = clf.predict_proba(test_inputs)
probs_west = []
probs_east = []

for i in range(len(probs)):
    
    team_id = team_ids[i]
    
    if team_id in west_teams["tmID"].values:
        probs_west.append((team_id, probs[i][1]))
        
    if team_id in east_teams["tmID"].values:
        probs_east.append((team_id, probs[i][1]))
    
    # print(f"{team_name[0]}: {probs[i][1]}")
    
probs_west = sorted(probs_west, key=lambda x: x[1], reverse=True)
probs_east = sorted(probs_east, key=lambda x: x[1], reverse=True)

print(probs_west)
print(probs_east)

### K-Nearest Neighbour

In [ ]:
# KNN

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=24)
knn.fit(train_inputs,train_labels)

y_pred = knn.predict(test_inputs)

#print metrics
print("Accuracy:",metrics.accuracy_score(test_labels,y_pred))
print("Precision:",metrics.precision_score(test_labels, y_pred, pos_label="Y"))
print("Recall:",metrics.recall_score(test_labels, y_pred, pos_label="Y"))
print("F1:",metrics.f1_score(test_labels, y_pred, pos_label="Y"))

# confusion matrix
confusion = confusion_matrix(test_labels, y_pred)
print(f"Confusion matrix:\n{confusion}")

#plot confusion matrix
plt.figure(figsize=(5,5))
sb.heatmap(confusion, annot=True, fmt="g", linewidths=.5, square = True, xticklabels=['No', 'Yes'], yticklabels=['No', 'Yes'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
all_sample_title = 'Passed to the playoffs?'

plt.title(all_sample_title, size = 10)
plt.show()

# print the probabilities for each class
probs = knn.predict_proba(test_inputs)
probs_west = []
probs_east = []

for i in range(len(probs)):
    
    team_id = team_ids[i]
    
    if team_id in west_teams["tmID"].values:
        probs_west.append((team_id, probs[i][1]))
        
    if team_id in east_teams["tmID"].values:
        probs_east.append((team_id, probs[i][1]))
    
    # print(f"{team_name[0]}: {probs[i][1]}")
    
probs_west = sorted(probs_west, key=lambda x: x[1], reverse=True)
probs_east = sorted(probs_east, key=lambda x: x[1], reverse=True)

print(probs_west)
print(probs_east)

### Decision Tree Classifier

In [ ]:
# Decision Tree

from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier()

dt.fit(train_inputs,train_labels)

#print decision tree
tree.plot_tree(dt)
columns = ['bioID', 'pos', 'height', 'weight', 'college', 'collegeOther',
       'birthDate', 'year', 'stint', 'tmID', 'points', 'oRebounds', 'dRebounds', 'rebounds',
       'assists', 'steals', 'blocks', 'turnovers', 'PF', 'fgAttempted',
       'fgMade', 'ftAttempted', 'ftMade', 'threeAttempted', 'threeMade']

y_pred = dt.predict(test_inputs)

#print metrics
print("Accuracy:",metrics.accuracy_score(test_labels,y_pred))
print("Precision:",metrics.precision_score(test_labels, y_pred, pos_label="Y"))
print("Recall:",metrics.recall_score(test_labels, y_pred, pos_label="Y"))
print("F1:",metrics.f1_score(test_labels, y_pred, pos_label="Y"))

# confusion matrix
confusion = confusion_matrix(test_labels, y_pred)
print(f"Confusion matrix:\n{confusion}")

#plot confusion matrix
plt.figure(figsize=(5,5))
sb.heatmap(confusion, annot=True, fmt="g", linewidths=.5, square = True, xticklabels=['No', 'Yes'], yticklabels=['No', 'Yes'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
all_sample_title = 'Passed to the playoffs?'

plt.title(all_sample_title, size = 10)
plt.show()

# print the probabilities for each class
probs = dt.predict_proba(test_inputs)
probs_west = []
probs_east = []

for i in range(len(probs)):
        
        team_id = team_ids[i]
        
        if team_id in west_teams["tmID"].values:
            probs_west.append((team_id, probs[i][1]))
            
        if team_id in east_teams["tmID"].values:
            probs_east.append((team_id, probs[i][1]))
        
        # print(f"{team_name[0]}: {probs[i][1]}")
        
probs_west = sorted(probs_west, key=lambda x: x[1], reverse=True)
probs_east = sorted(probs_east, key=lambda x: x[1], reverse=True)

# 4th try - The best model

Considering the data context we decided to develop a more complex model that considers teams as a set of players and coach.
Our goal is to make the final prediction based in the expected players performance according to their recorded statistics in the previous years.

In [ ]:
## create an empty dataframe without any column names, indices or data
df = pd.DataFrame()

## obtain the team IDS and year
team_ids = pd.read_sql("select tmID, year, confID from teams order by tmID", connection)

## iterate through each team and year
for index, row in team_ids.iterrows():

    ## obtain the team ID and year for each row
    team_id = row['tmID']
    year = row['year']
    confID = row['confID']

    # get all players from the team and year
    query = "select tmID, year, playerID from players_teams where tmID = '" + team_id + "' and year = " + str(year) + ";"

    ## obtain the players for each team
    team_players = pd.read_sql(query, connection)
    # points, oRebounds, dRebounds, rebounds, assists, steals, blocks, turnovers, PF, fgAttempted, fgMade, ftAttempted, ftMade, threeAttempted, threeMade
    team_stats = {"year": year, "points": 0, "oRebounds": 0, "dRebounds": 0, "rebounds": 0,
                  "assists": 0, "steals": 0, "blocks": 0, "turnovers": 0, "PF": 0, "fgAttempted": 0,
                  "fgMade": 0, "ftAttempted": 0, "ftMade": 0, "threeAttempted": 0, "threeMade": 0,
                  'weight': 0, "height": 0, "player_awards": 0, "confID": confID, "num_playoffs": 0,
                  "num_semis": 0, "num_finals": 0, "coach_win_ratio": 0, "coach_stint": 0,
                  "playoff": 0}

    # iterate through each player
    for idx, r in team_players.iterrows():
        ## obtain the player ID for each row
        player_id = r['playerID']

        # get the player's position
        query = "select pos from players where bioID = '" + player_id + "';"
        pos = pd.read_sql(query, connection).values[0][0]

        # get number of years played
        query = "select count(distinct year) as num_years from players_teams where playerID = '" + player_id + "' and year < " + str(year) + ";"
        num_years = pd.read_sql(query, connection).values[0][0]

        if num_years == 0:
            query = "select avg(points) as points, avg(oRebounds) as oRebounds, avg(dRebounds) as dRebounds, avg(rebounds) as rebounds, avg(assists) as assists, \
                avg(steals) as steals, avg(blocks) as blocks, avg(turnovers) as turnovers, avg(PF) as PF, avg(fgAttempted) as fgAttempted, \
                avg(fgMade) as fgMade, avg(ftAttempted) as ftAttempted, avg(ftMade) as ftMade, avg(threeAttempted) as threeAttempted, avg(threeMade) as threeMade \
                from players_teams join players on players_teams.playerID = players.bioID where year = " + str(year - 1) + " \
                and pos = '" + pos + "' and playerID not in (select playerID from players_teams where year < " + str(year - 1) + ");"
            player_stats = pd.read_sql(query, connection)

            ## add the player stats to the team stats
            team_stats['points'] += (player_stats['points'].values[0] or 0 )
            team_stats['oRebounds'] += (player_stats['oRebounds'].values[0] or 0 )
            team_stats['dRebounds'] += (player_stats['dRebounds'].values[0] or 0 )
            team_stats['rebounds'] += (player_stats['rebounds'].values[0] or 0 )
            team_stats['assists'] += (player_stats['assists'].values[0] or 0 )
            team_stats['steals'] += (player_stats['steals'].values[0] or 0 )
            team_stats['blocks'] += (player_stats['blocks'].values[0] or 0 )
            team_stats['turnovers'] += (player_stats['turnovers'].values[0] or 0 )
            team_stats['PF'] += (player_stats['PF'].values[0] or 0 )
            team_stats['fgAttempted'] += (player_stats['fgAttempted'].values[0] or 0 )
            team_stats['fgMade'] += (player_stats['fgMade'].values[0] or 0 )
            team_stats['ftAttempted'] += (player_stats['ftAttempted'].values[0] or 0 )
            team_stats['ftMade'] += (player_stats['ftMade'].values[0] or 0 )
            team_stats['threeAttempted'] += (player_stats['threeAttempted'].values[0] or 0 )
            team_stats['threeMade'] += (player_stats['threeMade'].values[0] or 0 )
            
        else:

            query = "select year, points, oRebounds, dRebounds, rebounds, assists, \
                    steals, blocks, turnovers, PF, fgAttempted, \
                    fgMade, ftAttempted, ftMade, threeAttempted, sum(threeMade) as threeMade \
                    from players_teams where playerID = '" + player_id + "' and year < " + str(year) + ";"
        
            ## obtain the average stats for each team
            player_stats = pd.read_sql(query, connection)

            points = oRebounds = dRebounds = rebounds = assists = steals = blocks = turnovers = PF = fgAttempted = fgMade = ftAttempted = ftMade = threeAttempted = threeMade = 0
            sum_weight = 0
            # iterate player stats and make weighted average for each year
            for j, row2 in player_stats.iterrows():

                iteration_year = row2['year']

                weight = 1 / (year - iteration_year)

                sum_weight += weight

                points += row2['points'] * weight
                oRebounds += row2['oRebounds'] * weight
                dRebounds += row2['dRebounds'] * weight
                rebounds += row2['rebounds'] * weight
                assists += row2['assists'] * weight
                steals += row2['steals'] * weight
                blocks += row2['blocks'] * weight
                turnovers += row2['turnovers'] * weight
                PF += row2['PF'] * weight
                fgAttempted += row2['fgAttempted'] * weight
                fgMade += row2['fgMade'] * weight
                ftAttempted += row2['ftAttempted'] * weight
                ftMade += row2['ftMade'] * weight
                threeAttempted += row2['threeAttempted'] * weight
                threeMade += row2['threeMade'] * weight

            # add the player stats to the team stats
            team_stats['points'] += points / sum_weight
            team_stats['oRebounds'] += oRebounds / sum_weight
            team_stats['dRebounds'] += dRebounds / sum_weight
            team_stats['rebounds'] += rebounds / sum_weight
            team_stats['assists'] += assists / sum_weight
            team_stats['steals'] += steals / sum_weight
            team_stats['blocks'] += blocks / sum_weight
            team_stats['turnovers'] += turnovers / sum_weight
            team_stats['PF'] += PF / sum_weight
            team_stats['fgAttempted'] += fgAttempted / sum_weight
            team_stats['fgMade'] += fgMade / sum_weight
            team_stats['ftAttempted'] += ftAttempted / sum_weight
            team_stats['ftMade'] += ftMade / sum_weight
            team_stats['threeAttempted'] += threeAttempted / sum_weight
            team_stats['threeMade'] += threeMade / sum_weight
        
        num_years = max(num_years, 1)


        # get each player num awards
        query = "select count(award) as num_awards_player from awards_players ap join players_teams pt on ap.year = pt.year \
                and ap.playerID = pt.playerID where ap.playerID = '" + player_id + "' and ap.year < " + str(year) + ";"
        num_awards_player = pd.read_sql(query, connection).values[0][0]
        team_stats['player_awards'] += num_awards_player

        # get the number of times the player went to the playoffs
        query = "select count(*) as num_playoffs from teams join players_teams on teams.year = players_teams.year and teams.tmID = players_teams.tmID\
            where playerID = '" + player_id + "' and teams.year < " + str(year) + " and firstRound <> '';"
        num_playoffs = pd.read_sql(query, connection).values[0][0]
        team_stats['num_playoffs'] += num_playoffs

        # get the number of times the player went to the semis
        query = "select count(*) as num_playoffs from teams join players_teams on teams.year = players_teams.year and teams.tmID = players_teams.tmID\
            where playerID = '" + player_id + "' and teams.year < " + str(year) + " and semis <> '';"
        num_semis = pd.read_sql(query, connection).values[0][0]
        team_stats['num_semis'] += num_semis

        # get the number of times the player went to the finals
        query = "select count(*) as num_playoffs from teams join players_teams on teams.year = players_teams.year and teams.tmID = players_teams.tmID\
            where playerID = '" + player_id + "' and teams.year < " + str(year) + " and finals <> '';"
        num_finals = pd.read_sql(query, connection).values[0][0]
        team_stats['num_finals'] += num_finals
    
    
    query = "select avg(weight), avg(height) from players_teams join players on players.bioID = players_teams.playerID \
        where tmID = '" + team_id + "' and year = " + str(year) + ";"
        
    ## obtain the average weight and height for each team
    team_weight_height = pd.read_sql(query, connection)
    
    ## add the average weight and height to the team stats
    team_stats['weight'] = team_weight_height['avg(weight)'].values[0]
    team_stats['height'] = team_weight_height['avg(height)'].values[0]

    # get team coach
    query = "select coachID, stint from coaches where tmID = '" + team_id + "' and year = "+ str(year) +";"
    coach_id = pd.read_sql(query, connection).values[0][0]
    stint = pd.read_sql(query, connection).values[0][1]
    team_stats['coach_stint'] = stint
    
    
    query = "select count(award) as num_awards from coaches join awards_players \
              on coaches.coachID = awards_players.playerID and coaches.year = awards_players.year \
              where coachID = '" + coach_id + "' and coaches.year < " + str(year) + ";"
    coach_awards = pd.read_sql(query, connection).values[0][0]
    team_stats['coach_awards'] = coach_awards

    # get average wins and losses from coach
    query = "select avg(won), avg(lost) from coaches where coachID = '" + coach_id + "' and year < " + str(year) + ";"
    coach_wins_losses = pd.read_sql(query, connection)
    num_matches_coach = (coach_wins_losses['avg(won)'].values[0] or 0) + (coach_wins_losses['avg(lost)'].values[0] or 0)
    if num_matches_coach == 0:
        team_stats['coach_win_ratio'] = 0
    else:
        team_stats['coach_win_ratio'] = (coach_wins_losses['avg(won)'].values[0] or 0) / num_matches_coach
        
        
    query = "select playoff from teams where tmID = '" + team_id + "' and year = " + str(year) + ";"
    playoff = pd.read_sql(query, connection)
    
    # check if it's Y or N
    if playoff['playoff'][0] == 'Y':
        team_stats['playoff'] = 1
    else:
        team_stats['playoff'] = 0
        
    
    # append the stats to the dataframe
    df = df._append(team_stats, ignore_index=True)


There is a entry for each team for each year. For each of this entries a team is represented by:
 - the sum of their players yearly statistics weighted average 
 - the average weight and height
 - the sum of players awards
 - the sum of the number of times the players reached playoffs, semis and finals
 - coach win ratio
 - conference the team belongs to

In [ ]:
categorical_columns = ["confID"]
for col in categorical_columns:
    df[col] = df[col].astype('category')

df= pd.get_dummies(df, columns=categorical_columns)

To train the model we need to split the data into training and testing sets. According to the context the division that makes sense is by years.

In [ ]:
target_year = 10

# # get all rows from df where year = target year
test_data = df.loc[df["year"] == target_year]

# # get all rows from df where year <> target_year and year <> 1
train_data = df.loc[df["year"] < target_year]
train_data = train_data.loc[train_data["year"] != 1]

labels = ['playoff']

inputs = []

for col in df.columns:
    if col not in labels:
        inputs.append(col)

train_inputs = train_data[inputs].values
train_labels = train_data[labels].values

test_inputs = test_data[inputs].values
test_labels = test_data[labels].values


Considering the features are in different scales it is a good practice to standardize it into a range from 0 to 1. This can be achieved using the MinMaxScaler.

In [ ]:
# min max scaler
from sklearn.preprocessing import StandardScaler, MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(train_inputs)

train_inputs = scaler.transform(train_inputs)
test_inputs = scaler.transform(test_inputs)

Now that everything is set we can start applying the algorithms.

### Logistic Regression

In [ ]:
# logistic regression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(solver='liblinear', random_state=0)
logreg.fit(train_inputs,train_labels)

y_pred = logreg.predict(test_inputs)

In the model testing phase it is useful to gather some metrics relative to the way it is working and the quality of its predictions.

With this in mind we considered relevant metrics:
 - Accuracy
 - Precision
 - Recall
 - F1
 - Confusion Matrix
 - ROC
 - AUC

(Note: this will only work when testing the model, so the target year variable should be set to 10. When the target year is 11 then there will be no test labels to compare with the predictions.)

In [ ]:
#print metrics
print("Accuracy:",metrics.accuracy_score(test_labels,y_pred))
print("Precision:",metrics.precision_score(test_labels, y_pred))
print("Recall:",metrics.recall_score(test_labels, y_pred))
print("F1:",metrics.f1_score(test_labels, y_pred))

feature_importance = abs(logreg.coef_[0])
feature_importance = sorted(zip(inputs, feature_importance), key=lambda x: x[1], reverse=True)

# print("Feature importance:")
# for i in range(len(feature_importance)):
#     print(f"{feature_importance[i][0]}: {feature_importance[i][1]}")

# confusion matrix
confusion = confusion_matrix(test_labels, y_pred)
print(f"Confusion matrix:\n{confusion}")

#plot confusion matrix
plt.figure(figsize=(5,5))
sb.heatmap(confusion, annot=True, fmt="g", linewidths=.5, square = True, xticklabels=['No', 'Yes'], yticklabels=['No', 'Yes'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
all_sample_title = 'Passed to the playoffs?'
plt.title(all_sample_title, size = 10)
plt.show()

# roc curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

logit_roc_auc = roc_auc_score(test_labels, logreg.predict(test_inputs))
fpr, tpr, thresholds = roc_curve(test_labels, logreg.predict_proba(test_inputs)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0,1], [0,1], 'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver operating characteristic")
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

Since the algorithm does not know that only 4 teams from each conference qualify to the playoffs we need to enforce this knowledge in our predictions. So, we divide the teams by conference, and consider that they qualify if they are one of the 4 most likely to, even if the probability is below 0.5.

In [ ]:
query = "select tmId, confID from teams where year = " + str(target_year) + ";"
teams_conf_ids = pd.read_sql(query, connection)

query = "select tmID from teams where year = " + str(target_year) + ";"

# print(teams_conf_ids)

west_teams = teams_conf_ids[teams_conf_ids["confID"] == "WE"]
east_teams = teams_conf_ids[teams_conf_ids["confID"] == "EA"]

# print(west_teams)
# print(east_teams)

# print the probabilities for each class
probs = logreg.predict_proba(test_inputs)
probs_west = []
probs_east = []

query = "select tmID from teams where year = " + str(target_year) + ";"
team_ids = pd.read_sql(query, connection)
team_ids = team_ids["tmID"].values

for i in range(len(probs)):
    team_id = team_ids[i]
    
    if team_id in west_teams["tmID"].values:
        probs_west.append((team_id, probs[i][1]))
        
    if team_id in east_teams["tmID"].values:
        probs_east.append((team_id, probs[i][1]))
    
    # print(f"{team_name[0]}: {probs[i][1]}")

probs_west = sorted(probs_west, key=lambda x: x[1], reverse=True)
probs_east = sorted(probs_east, key=lambda x: x[1], reverse=True)

print(probs_west)
print(probs_east)

Finally we have the logistic regression predictions.

Now we can train and predict using different algorithms.

### Neural Network

In [ ]:
# neural network

from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(100, 100, 100), max_iter=2000, alpha=0.0001, solver='adam', verbose=0, random_state=21, tol=0.000000001)
mlp.fit(train_inputs,train_labels)

y_pred = mlp.predict(test_inputs)

In [ ]:
#print metrics
print("Accuracy:",metrics.accuracy_score(test_labels,y_pred))
print("Precision:",metrics.precision_score(test_labels, y_pred))

print("Recall:",metrics.recall_score(test_labels, y_pred))
print("F1:",metrics.f1_score(test_labels, y_pred))

# confusion matrix
confusion = confusion_matrix(test_labels, y_pred)
print(f"Confusion matrix:\n{confusion}")

#plot confusion matrix
plt.figure(figsize=(5,5))
sb.heatmap(confusion, annot=True, fmt="g", linewidths=.5, square = True, xticklabels=['No', 'Yes'], yticklabels=['No', 'Yes'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
all_sample_title = 'Passed to the playoffs?'
plt.title(all_sample_title, size = 10)
plt.show()

# roc curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

neural_net_roc_auc = roc_auc_score(test_labels, mlp.predict(test_inputs))
fpr, tpr, thresholds = roc_curve(test_labels, mlp.predict_proba(test_inputs)[:,1])
plt.figure()

plt.plot(fpr, tpr, label='Neural Network (area = %0.2f)' % neural_net_roc_auc)
plt.plot([0,1], [0,1], 'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])

plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver operating characteristic")
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()


In [ ]:
# print the probabilities for each class
probs = mlp.predict_proba(test_inputs)
probs_west = []
probs_east = []

for i in range(len(probs)):
    team_id = team_ids[i]
    
    if team_id in west_teams["tmID"].values:
        probs_west.append((team_id, probs[i][1]))
        
    if team_id in east_teams["tmID"].values:
        probs_east.append((team_id, probs[i][1]))
    
    # print(f"{team_name[0]}: {probs[i][1]}")
    
probs_west = sorted(probs_west, key=lambda x: x[1], reverse=True)
probs_east = sorted(probs_east, key=lambda x: x[1], reverse=True)

print(probs_west)
print(probs_east)

### K-Nearest Neighbours

In [ ]:
#standardize data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(train_inputs)

train_inputs = scaler.transform(train_inputs)
test_inputs = scaler.transform(test_inputs)

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=7, weights='distance')
knn.fit(train_inputs,train_labels)

y_pred = knn.predict(test_inputs)

#print metrics
print("Accuracy:",metrics.accuracy_score(test_labels,y_pred))
print("Precision:",metrics.precision_score(test_labels, y_pred))
print("Recall:",metrics.recall_score(test_labels, y_pred))
print("F1:",metrics.f1_score(test_labels, y_pred))

# confusion matrix
confusion = confusion_matrix(test_labels, y_pred)
print(f"Confusion matrix:\n{confusion}")

#plot confusion matrix
plt.figure(figsize=(5,5))
sb.heatmap(confusion, annot=True, fmt="g", linewidths=.5, square = True, xticklabels=['No', 'Yes'], yticklabels=['No', 'Yes'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
all_sample_title = 'Passed to the playoffs?'

plt.title(all_sample_title, size = 10)
plt.show()

# roc curve

from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

knn_roc_auc = roc_auc_score(test_labels, knn.predict(test_inputs))  
fpr, tpr, thresholds = roc_curve(test_labels, knn.predict_proba(test_inputs)[:,1])
plt.figure()

plt.plot(fpr, tpr, label='KNN (area = %0.2f)' % knn_roc_auc)
plt.plot([0,1], [0,1], 'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])

plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver operating characteristic")
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

# print the probabilities for each class
probs = knn.predict_proba(test_inputs)
probs_west = []
probs_east = []

for i in range(len(probs)):
    
    team_id = team_ids[i]
    
    if team_id in west_teams["tmID"].values:
        probs_west.append((team_id, probs[i][1]))
        
    if team_id in east_teams["tmID"].values:
        probs_east.append((team_id, probs[i][1]))
    
    # print(f"{team_name[0]}: {probs[i][1]}")
    
probs_west = sorted(probs_west, key=lambda x: x[1], reverse=True)
probs_east = sorted(probs_east, key=lambda x: x[1], reverse=True)

print(probs_west)
print(probs_east)

### Support Vector Machines (SVM)

In [ ]:
# SVM

from sklearn import svm

clf = svm.SVC(kernel='linear', C=1, probability=True)
clf.fit(train_inputs,train_labels)

y_pred = clf.predict(test_inputs)

#print metrics
print("Accuracy:",metrics.accuracy_score(test_labels,y_pred))
print("Precision:",metrics.precision_score(test_labels, y_pred))
print("Recall:",metrics.recall_score(test_labels, y_pred))
print("F1:",metrics.f1_score(test_labels, y_pred))

# confusion matrix
confusion = confusion_matrix(test_labels, y_pred)
print(f"Confusion matrix:\n{confusion}")

#plot confusion matrix
plt.figure(figsize=(5,5))
sb.heatmap(confusion, annot=True, fmt="g", linewidths=.5, square = True, xticklabels=['No', 'Yes'], yticklabels=['No', 'Yes'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
all_sample_title = 'Passed to the playoffs?'

plt.title(all_sample_title, size = 10)
plt.show()

# roc curve

from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

svm_roc_auc = roc_auc_score(test_labels, clf.predict(test_inputs))
fpr, tpr, thresholds = roc_curve(test_labels, clf.predict_proba(test_inputs)[:,1])
plt.figure()

plt.plot(fpr, tpr, label='SVM (area = %0.2f)' % svm_roc_auc)
plt.plot([0,1], [0,1], 'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])

plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver operating characteristic")
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

# print the probabilities for each class
probs = clf.predict_proba(test_inputs)
probs_west = []
probs_east = []

for i in range(len(probs)):
        
        team_id = team_ids[i]
        
        if team_id in west_teams["tmID"].values:
            probs_west.append((team_id, probs[i][1]))
            
        if team_id in east_teams["tmID"].values:
            probs_east.append((team_id, probs[i][1]))
        
        # print(f"{team_name[0]}: {probs[i][1]}")
        
probs_west = sorted(probs_west, key=lambda x: x[1], reverse=True)
probs_east = sorted(probs_east, key=lambda x: x[1], reverse=True)

print(probs_west)
print(probs_east)